# --- Day 12: Hill Climbing Algorithm ---

You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

Sabqponm <br>
abcryxxl <br>
accszExk <br>
acctuvwj <br>
abdefghi <br>

Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:

v..v<<<< <br>
\>v.vv<<^ <br>
.>vv>E^^ <br>
..v>>>^^ <br>
..>>>>>^ <br>

In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?

In [1]:
import networkx as nx
def read_input(filename):
    terrain = []
    with open(filename) as file:        
        for line in file:
            row = [c for c in line if c != '\n']
            if row:
                terrain.append(row)
    return terrain

def get_height(symbol):
    if symbol == 'S':
        height = 0
    elif symbol == 'E':
        height = ord('z') - ord('a')
    else:
        height = ord(symbol) - ord('a')
    return height

def create_graph(terrain):
    G = nx.DiGraph()
    start = ''
    end = ''
    for i in range(len(terrain)):
        for j in range(len(terrain[0])):
            if terrain[i][j] == 'S':
                start = f'({i},{j})'
            elif terrain[i][j] == 'E':
                end = f'({i},{j})'
                
            init_height = get_height(terrain[i][j])            
            #left
            if j - 1 >= 0:
                curr_height = get_height(terrain[i][j-1])
                if curr_height - 1 <= init_height:
                    G.add_edge(f'({i},{j})', f'({i},{j-1})')
            #right
            if j + 1 < len(terrain[0]):
                curr_height = get_height(terrain[i][j+1])
                if curr_height - 1 <= init_height:
                    G.add_edge(f'({i},{j})', f'({i},{j+1})')
            #top
            if i - 1 >= 0:
                curr_height = get_height(terrain[i-1][j])
                if curr_height - 1 <= init_height:
                    G.add_edge(f'({i},{j})', f'({i-1},{j})')
            #down
            if i + 1 < len(terrain):
                curr_height = get_height(terrain[i+1][j])
                if curr_height - 1 <= init_height:
                    G.add_edge(f'({i},{j})', f'({i+1},{j})')
            
    return start, end, G

In [2]:
terrain = read_input('test.txt')
start, end, terrain_graph = create_graph(terrain)
path = nx.shortest_path(terrain_graph, source=start, target=end)
print(len(path)-1)

31


In [3]:
terrain = read_input('input.txt')
start, end, terrain_graph = create_graph(terrain)
path = nx.shortest_path(terrain_graph, source=start, target=end)
print(len(path)-1)

520


# --- Part Two ---

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:

Sabqponm <br>
abcryxxl <br>
accszExk <br>
acctuvwj <br>
abdefghi <br>

Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:

...v<<<< <br>
...vv<<^ <br>
...v>E^^ <br>
.>v>>>^^ <br>
\>^>>>>>^ <br>

his path reaches the goal in only 29 steps, the fewest possible.

What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?

In [9]:
def solve(filename):
    terrain = read_input(filename)
    start, end, terrain_graph = create_graph(terrain)
    min_path = nx.shortest_path(terrain_graph, source=start, target=end)
    min_length = len(min_path) - 1
    
    shortest_paths = dict(nx.single_target_shortest_path(terrain_graph, end, min_length))
    
    for i in range(len(terrain)):
        for j in range(len(terrain[0])):
            if terrain[i][j] == 'a':
                curr_path = shortest_paths.get(f'({i},{j})', None)
                if curr_path and len(curr_path) < min_length:
                    min_length = len(curr_path) - 1
    return min_length

In [11]:
solve('test.txt')

29

In [12]:
solve('input.txt')

508